# Set up

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow
print(tensorflow.__version__)

In [0]:
%load_ext tensorboard

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

from datetime import datetime

from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


# Import Data

In [0]:
fashion_mnist = keras.datasets.fashion_mnist
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
CLASS_NAMES = np.array(['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'])
CLASS_NAMES


In [0]:
train_data = tf.cast(train_data, tf.float32)

train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(train_data.shape[0])
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels)).shuffle(test_data.shape[0])

print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

In [0]:
def show_batch(ds):
  plt.figure(figsize=(10,10))
  i = 0
  for data, label in ds.take(25):
      ax = plt.subplot(5,5,i+1)
      plt.imshow(data)
      plt.title(CLASS_NAMES[label])
      plt.axis('off')
      i = i+1

show_batch(test_dataset)     

# Train

In [0]:
class Classifier(tf.keras.Model):
  def __init__(self, num_units=64, conv_filter=32, dropout=0.1, num_classes=10):
    super(Classifier, self).__init__(name='classifier')
    self.num_classes = num_classes
    self.conv = keras.layers.Conv2D(conv_filter, (3, 3), activation='relu', padding='valid')
    self.max_pool = keras.layers.MaxPooling2D(pool_size=(2, 2))
    self.flatten = keras.layers.Flatten()
    self.dense_1 = keras.layers.Dense(num_units, activation='relu')
    self.dropout = keras.layers.Dropout(dropout)
    self.dense_2 = keras.layers.Dense(num_classes, activation='softmax')
  def call(self, inputs):
    # Define your forward pass here,
    # using layers you previously defined (in `__init__`).
    inputs = tf.expand_dims(inputs, 3)
    x = self.conv(inputs)
    x = self.max_pool(x)
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.dropout(x)
    return self.dense_2(x)

In [0]:
def train(num_units=64, conv_filter=32, dropout=0.1, batch_size=32, epochs=5, num_classes=10):
  train_ds = train_dataset.batch(batch_size)
  test_ds = test_dataset.batch(batch_size)
  classifier = Classifier(num_units=num_units, 
                          conv_filter=conv_filter,
                          dropout=dropout, 
                          num_classes=num_classes)
  classifier.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
  classifier.fit(train_ds, callbacks=[tensorboard_callback], epochs=epochs)
  _, accuracy = classifier.evaluate(test_ds)
  return accuracy


In [0]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
train(num_units=128)

In [0]:
%tensorboard --logdir logs/scalars

# Hyper Parameters Tunning

In [0]:
!rm -rf ./logs/

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32, 64]))
HP_CONV_FILTER = hp.HParam('conv_filter', hp.Discrete([32]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1]))
HP_EPOCHS = hp.HParam('ephchs', hp.Discrete([5, 10]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32, 64]))
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_CONV_FILTER, HP_DROPOUT, HP_EPOCHS, HP_BATCH_SIZE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)
    accuracy = train(num_units=hparams[HP_NUM_UNITS],
                     conv_filter=hparams[HP_CONV_FILTER],
                     dropout=hparams[HP_DROPOUT],
                     batch_size=hparams[HP_BATCH_SIZE],
                     epochs=hparams[HP_EPOCHS])
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

session_num = 0
for num_units in HP_NUM_UNITS.domain.values:
  for conv_filter in HP_CONV_FILTER.domain.values:
    for dropout in HP_DROPOUT.domain.values:
      for epochs in HP_EPOCHS.domain.values:
        for batch_size in HP_BATCH_SIZE.domain.values:
          hparams = {
              HP_NUM_UNITS: num_units,
              HP_CONV_FILTER: conv_filter,
              HP_DROPOUT: dropout,
              HP_EPOCHS: epochs,
              HP_BATCH_SIZE: batch_size,
              }
          run_name = "run-%d" % session_num
          print('--- Starting trial: %s' % run_name)
          print({h.name: hparams[h] for h in hparams})
          run('logs/hparam_tuning/' + run_name, hparams)
          session_num += 1

In [0]:
%tensorboard --logdir logs/hparam_tuning

# Predict

In [0]:
import random 
  
idx = random.randint(0, test_data.shape[0]) 
validation_data = test_data[idx:idx+1]
print(validation_data.shape)
predictions = classifier.predict(validation_data)
print(predictions.shape)
pred = CLASS_NAMES[np.argmax(predictions)]
want = CLASS_NAMES[test_labels[idx]]
print("pred: ", pred)
print("want: ", want)

#Reference

*   https://www.tensorflow.org/guide/keras/train_and_evaluate#training_evaluation_from_tfdata_datasets
*   https://gist.github.com/datlife/abfe263803691a8864b7a2d4f87c4ab8
*   https://lambdalabs.com/blog/tensorflow-2-0-tutorial-01-image-classification-basics/
*   https://keras.io/layers/convolutional/
*   https://github.com/keras-team/keras/issues/3385

